# Predict Future Sales
from [kaggle competition](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/)

## 1. Early Data Preprocessing

You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

### File Description
* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
* sales_test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
* sample_submission.csv - a sample submission file in the correct format.
* items.csv - supplemental information about the items/products.
* item_categories.csv  - supplemental information about the items categories.
* shops.csv- supplemental information about the shops.


### Data Fields
* ID - an Id that represents a (Shop, Item) tuple within the test set
* shop_id - unique identifier of a shop
* item_id - unique identifier of a product
* item_category_id - unique identifier of item category
* item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
* item_price - current price of an item
* date - date in format dd/mm/yyyy
* date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* item_name - name of item
* shop_name - name of shop
* item_category_name - name of item category

In [57]:
import pandas as pd
import numpy as np

In [58]:
train = pd.read_csv('data/sales_train.csv')
test = pd.read_csv('data/sales_test.csv')
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [59]:
items = pd.read_csv('data/items.csv')
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [60]:
item_categories = pd.read_csv('data/item_categories.csv')
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [61]:
shops = pd.read_csv('data/shops.csv')
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


## Re-format the dataframe

Since the prediction goal of this project is to predict the monthly revenues in `Nov. 2015` for each store and each item using sales records from `Jan. 2013` to `Oct. 2015`. However the current format of dataset seems inappropriate to model.<br>

Before we optimize and pre-process the dataset, we will group the dataset by `store_id` and `item_id` to re-format the dataset with monthly data.<br>

* `AS WAS` : sales recorded daily grouped by `shop_id` + `item_id`
* `TO BE` : sales recorded monthly grouped by `shop_id` + `item_id`

### Target column = ?

Final goal is to predict the monthly revenue grouped by `store_id` and `item_id` - however, the dataset currently does not contain the revenue values. Instead, it contains the number of daily sold and the price the item sold at.<br>

At this early stage, we are not going to fix the approach to get the goal - we just convert the format only to represent records from `daily` to `monthly`.

In [62]:
print(len(train['shop_id'].unique()))
print(len(train['item_id'].unique()))

60
21807


In [63]:
train['date'].apply(lambda x: x[3:]).unique()

array(['01.2013', '02.2013', '03.2013', '04.2013', '05.2013', '06.2013',
       '07.2013', '08.2013', '09.2013', '10.2013', '11.2013', '12.2013',
       '01.2014', '02.2014', '03.2014', '04.2014', '05.2014', '06.2014',
       '07.2014', '08.2014', '09.2014', '10.2014', '11.2014', '12.2014',
       '01.2015', '02.2015', '03.2015', '04.2015', '05.2015', '06.2015',
       '07.2015', '08.2015', '09.2015', '10.2015'], dtype=object)

In [64]:
# total item_cnt_day grouped by shop_id - item_id - date_block_num
# convert the groupby object to dataframe
train_groupby_item_cnt = train.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum().reset_index()
train_groupby_item_cnt[:10]

,shop_id,item_id,date_block_num,item_cnt_day
0,0,30,1,31.0
1,0,31,1,11.0
2,0,32,0,6.0
3,0,32,1,10.0
4,0,33,0,3.0
5,0,33,1,3.0
6,0,35,0,1.0
7,0,35,1,14.0
8,0,36,1,1.0
9,0,40,1,1.0


In [65]:
# average item price column grouped by shop_id - item_id - date_block_num
# (since product prices can vary in month/ store)
# convert the groupby object to dataframe

train_groupby_item_price = train.groupby(['shop_id', 'item_id', 'date_block_num'])['item_price'].mean().reset_index()
train_groupby_item_price[:10]

,shop_id,item_id,date_block_num,item_price
0,0,30,1,265.0
1,0,31,1,434.0
2,0,32,0,221.0
3,0,32,1,221.0
4,0,33,0,347.0
5,0,33,1,347.0
6,0,35,0,247.0
7,0,35,1,247.0
8,0,36,1,357.0
9,0,40,1,127.0


In [66]:
# concatenate two dataframe and remove duplicated columns
# and change the column name

columns_to_change = {
    'shop_id':'shop_id',
    'item_id':'item_id',
    'date_block_num':'date_block_num',
    'item_cnt_day':'item_cnt_month',
    'item_price':'item_price_month'    
}

train_month = pd.concat([train_groupby_item_cnt, train_groupby_item_price['item_price']],
                         axis=1).rename(columns=columns_to_change)

train_month.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price_month
0,0,30,1,31.0,265.0
1,0,31,1,11.0,434.0
2,0,32,0,6.0,221.0
3,0,32,1,10.0,221.0
4,0,33,0,3.0,347.0


In [67]:
# and add timestamp column in reference to 'date_block_num' column

blocknum_monthyear_mapped = dict(zip(train['date_block_num'],
                                 train['date'].apply(lambda x: x[3:])))

In [68]:
train_month['date_month'] = train_month['date_block_num'].map(blocknum_monthyear_mapped).apply(lambda x: x[:2])
train_month['date_year'] = train_month['date_block_num'].map(blocknum_monthyear_mapped).apply(lambda x: x[3:])

In [69]:
train_month.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price_month,date_month,date_year
0,0,30,1,31.0,265.0,02,2013
1,0,31,1,11.0,434.0,02,2013
2,0,32,0,6.0,221.0,01,2013
3,0,32,1,10.0,221.0,02,2013
4,0,33,0,3.0,347.0,01,2013


In [70]:
# update train variable
train = train_month.iloc[:,:]

## Concatenate & combine data


In [71]:
# map the 'item_category_id' column values from 'item' dataframe
# and change the type as category

item_item_cat_id_mapped = dict(zip(items['item_id'].tolist(), items['item_category_id'].tolist()))
train['item_category_id'] = train['item_id'].map(item_item_cat_id_mapped)

In [72]:
train['item_category_id'] = train['item_category_id'].astype('category')  

In [73]:
# map the 'item_category_name' column values from 'item_category_name'
# we do not change 'item_category_name' in train dataframe at now

item_cat_id_item_name_mapped = dict(zip(item_categories['item_category_id'].tolist(),
                                       item_categories['item_category_name'].tolist()))
train['item_category_name'] = train['item_category_id'].map(item_cat_id_item_name_mapped)

In [74]:
train.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price_month,date_month,date_year,item_category_id,item_category_name
0,0,30,1,31.0,265.0,02,2013,40,Кино - DVD
1,0,31,1,11.0,434.0,02,2013,37,Кино - Blu-Ray
2,0,32,0,6.0,221.0,01,2013,40,Кино - DVD
3,0,32,1,10.0,221.0,02,2013,40,Кино - DVD
4,0,33,0,3.0,347.0,01,2013,37,Кино - Blu-Ray


## Get additional data using `Google API`
* **Translate Russian text in `item_category_name` into English text using Google Translator**
* **Get the full address of the store by searching the `shop_name` via Google Searching**

In [76]:
# Get translated item_category_name as English

item_category_names = train['item_category_name'].unique()
len(item_category_names)

84

In [77]:
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    '/Users/choigww/*.json')

In [78]:
# Imports the Google Cloud client library
from google.cloud import translate
translate_client = translate.Client(credentials=credentials)

In [79]:
translated_item_category_names = []

for unique_cat_name in item_category_names:
    # The text to translate
    text = unique_cat_name

    # The target language
    target = 'en'

    # Translates some text into Russian
    translation = translate_client.translate(
        text,
        target_language=target)

    #print(u'Text: {}'.format(text))
    #print(u'Translation: {}'.format(translation['translatedText']))
    translated_item_category_names.append(translation['translatedText'])

In [80]:
# Check; all texts translated successfully?
print(translated_item_category_names)

['Cinema - DVD', 'Cinema - Blu-Ray', 'Music - MP3', 'Books - Audiobooks', 'Books - Audiobooks 1C', 'Books - Methodical materials 1С', 'Programs - 1C: Enterprise 8', 'Programs - Teaching', 'PC Games - Standard Edition', 'Music - CD of local production', 'Gifts - Development', 'Music - Musical video', 'Programs - Home and Office', 'PC Games - Collector&#39;s Edition', 'PC Games - Additional Edition', 'Games - PS3', 'Games - XBOX 360', 'Games - PSVita', 'Games - PSP', 'Accessories - PSP', 'Music - Gift edition', 'Gifts - Gadgets, robots, sports', 'Music - CD of branded production', 'Game Consoles - XBOX 360', 'Accessories - PS3', 'Accessories - PSVita', 'Accessories - PS2', 'Payment cards - PSN', 'Games - Accessories for games', 'Game Consoles - PSVita', 'Game Consoles - PSP', 'Game Consoles - PS3', 'Gifts - Souvenirs', 'Accessories - XBOX 360', 'Payment cards - Live!', 'Cinema - Blu-Ray 3D', 'Cinema - Collector&#39;s', 'Gifts - Board games (compact)', 'Gifts - Soft toys', 'Gifts - Souven

## Text column preprocessing

* Validate the translation done successfully
  * If not, execute translation again
* transform as lowercase
* Remove unnecessary characters
* and?

In [81]:
translated_item_category_names.index('Книги - Путеводители')

63

In [82]:
translation = translate_client.translate(
    translated_item_category_names[63],
    target_language=target)

translation['translatedText']

'Книги - Путеводители'

In [83]:
# google translate api does not work for this case
# manullay find the translated word in English

translated_item_category_names[63] = 'Books - Travel Guides'

In [84]:
print(translated_item_category_names)

['Cinema - DVD', 'Cinema - Blu-Ray', 'Music - MP3', 'Books - Audiobooks', 'Books - Audiobooks 1C', 'Books - Methodical materials 1С', 'Programs - 1C: Enterprise 8', 'Programs - Teaching', 'PC Games - Standard Edition', 'Music - CD of local production', 'Gifts - Development', 'Music - Musical video', 'Programs - Home and Office', 'PC Games - Collector&#39;s Edition', 'PC Games - Additional Edition', 'Games - PS3', 'Games - XBOX 360', 'Games - PSVita', 'Games - PSP', 'Accessories - PSP', 'Music - Gift edition', 'Gifts - Gadgets, robots, sports', 'Music - CD of branded production', 'Game Consoles - XBOX 360', 'Accessories - PS3', 'Accessories - PSVita', 'Accessories - PS2', 'Payment cards - PSN', 'Games - Accessories for games', 'Game Consoles - PSVita', 'Game Consoles - PSP', 'Game Consoles - PS3', 'Gifts - Souvenirs', 'Accessories - XBOX 360', 'Payment cards - Live!', 'Cinema - Blu-Ray 3D', 'Cinema - Collector&#39;s', 'Gifts - Board games (compact)', 'Gifts - Soft toys', 'Gifts - Souven

#### We can split each category name by ` - `, however;
* Many word seem to have been translated as `number`, `figure`, `numeral`, etc ...
  * we doubt whether the translation is correct or not.
  * **We consider them as `digital` type purchase and keep them in data.**
* There are some inevident categorization
  * For the first categry name, `'payment cards'` and `'payment cards ((movies, music, games))'` exist seperately
  * `official - tickets` and `tickets (figure)` as the first category names exist seperately as well
    * **For `payment cards`, we can combine them. For the `tickets` thing, we just leave them as divided.**


In [85]:
import re

In [86]:
translated_item_category_names_dict = {}

for i, word in enumerate(translated_item_category_names):
    
    word = word.lower()
    split = word.split(' - ')
    
    first_cat = split[0]
    
    try:
        second_cat = split[1]
    except:
        second_cat = ''
        
    if first_cat not in translated_item_category_names_dict:
        translated_item_category_names_dict[first_cat] = []
        
    translated_item_category_names_dict[first_cat].append(second_cat)
    
translated_item_category_names_dict

{'accessories': ['psp', 'ps3', 'psvita', 'ps2', 'xbox 360', 'ps4', 'xbox one'],
 'android games': ['digit'],
 'books': ['audiobooks',
  'audiobooks 1c',
  'methodical materials 1с',
  'artbook, encyclopedia',
  'comics, manga',
  'travel guides',
  'business literature',
  'fiction',
  'postcards',
  'computer literature',
  'cognitive literature',
  'number',
  'audiobooks (numbers)'],
 'cinema': ['dvd', 'blu-ray', 'blu-ray 3d', 'collector&#39;s'],
 'clean carriers (spire)': [''],
 'clean media (piece)': [''],
 'delivery of goods': [''],
 'elements of a food': [''],
 'game consoles': ['xbox 360',
  'psvita',
  'psp',
  'ps3',
  'ps4',
  'xbox one',
  'other',
  'ps2'],
 'games': ['ps3',
  'xbox 360',
  'psvita',
  'psp',
  'accessories for games',
  'ps4',
  'xbox one',
  'ps2'],
 'gifts': ['development',
  'gadgets, robots, sports',
  'souvenirs',
  'board games (compact)',
  'soft toys',
  'souvenirs (per sample)',
  'board games',
  'figures',
  'bags, albums, mouse pads',
  'attri

In [87]:
# reflect the above

for i, word in enumerate(translated_item_category_names):
    
    word_ = re.sub(r'figure|numbers|number|numeral|digital|digit', 'digital', word.lower())
    translated_item_category_names[i] = word_
    
    if word_ == 'payment cards (movies, music, games)':
        translated_item_category_names[i] = 'payment cards - movies, music, games'
    
    print(translated_item_category_names[i])

cinema - dvd
cinema - blu-ray
music - mp3
books - audiobooks
books - audiobooks 1c
books - methodical materials 1с
programs - 1c: enterprise 8
programs - teaching
pc games - standard edition
music - cd of local production
gifts - development
music - musical video
programs - home and office
pc games - collector&#39;s edition
pc games - additional edition
games - ps3
games - xbox 360
games - psvita
games - psp
accessories - psp
music - gift edition
gifts - gadgets, robots, sports
music - cd of branded production
game consoles - xbox 360
accessories - ps3
accessories - psvita
accessories - ps2
payment cards - psn
games - accessories for games
game consoles - psvita
game consoles - psp
game consoles - ps3
gifts - souvenirs
accessories - xbox 360
payment cards - live!
cinema - blu-ray 3d
cinema - collector&#39;s
gifts - board games (compact)
gifts - soft toys
gifts - souvenirs (per sample)
accessories - ps4
gifts - board games
clean media (piece)
clean carriers (spire)
pc - headsets / headp

In [88]:
translated_catname_mapped = dict(zip(item_category_names,
                                    translated_item_category_names))

train['item_category_name_eng'] = train['item_category_name'].map(translated_catname_mapped)
train = train.drop(['item_category_name'], axis=1)

In [89]:
train.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price_month,date_month,date_year,item_category_id,item_category_name_eng
0,0,30,1,31.0,265.0,02,2013,40,cinema - dvd
1,0,31,1,11.0,434.0,02,2013,37,cinema - blu-ray
2,0,32,0,6.0,221.0,01,2013,40,cinema - dvd
3,0,32,1,10.0,221.0,02,2013,40,cinema - dvd
4,0,33,0,3.0,347.0,01,2013,37,cinema - blu-ray


In [90]:
# Get translated shop_name as English
# Since we did not execute `id-name` mapping regarding shop, 

shop_id_item_name_mapped = dict(zip(shops['shop_id'].tolist(),
                                       shops['shop_name'].tolist()))
train['shop_name'] = train['shop_id'].map(shop_id_item_name_mapped)

In [91]:
shop_names = train['shop_name'].unique()
len(shop_names)

60

### As above, we gather translations, validate and reflect to the `shop_name` data.

In [92]:
shop_loc_names = train['shop_name'].apply(lambda x: x.split(' ')[0])
train['shop_loc_name'] = shop_loc_names

In [93]:
train.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price_month,date_month,date_year,item_category_id,item_category_name_eng,shop_name,shop_loc_name
0,0,30,1,31.0,265.0,02,2013,40,cinema - dvd,"!Якутск Орджоникидзе, 56 фран",!Якутск
1,0,31,1,11.0,434.0,02,2013,37,cinema - blu-ray,"!Якутск Орджоникидзе, 56 фран",!Якутск
2,0,32,0,6.0,221.0,01,2013,40,cinema - dvd,"!Якутск Орджоникидзе, 56 фран",!Якутск
3,0,32,1,10.0,221.0,02,2013,40,cinema - dvd,"!Якутск Орджоникидзе, 56 фран",!Якутск
4,0,33,0,3.0,347.0,01,2013,37,cinema - blu-ray,"!Якутск Орджоникидзе, 56 фран",!Якутск


In [94]:
shop_loc_names_uniques = train['shop_loc_name'].unique().tolist()
shop_loc_names_translated = []

for unique_shop_loc_name in shop_loc_names_uniques:
    text = unique_shop_loc_name
    target = 'en'
    translation = translate_client.translate(
        text,
        target_language=target)

    shop_loc_names_translated.append(translation['translatedText'].lower())

In [95]:
print(shop_loc_names_uniques)

['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж', 'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга', 'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород', 'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев', 'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов', 'Якутск', 'Ярославль']


In [96]:
print(shop_loc_names_translated)

['yakutsk', 'adygea', 'balashiha', 'volzhsky', 'vologda', 'voronezh', 'traveling', 'zhukovsky', 'online store', 'kazan', 'kaluga', 'kolomna', 'krasnoyarsk', 'kursk', 'moscow', 'mytischi', 'n.novgorod', 'novosibirsk', 'omsk', 'rostovnadonu', 'st. petersburg', 'samara', 'sergiev', 'surgut', 'tomsk', 'tyumen', 'ufa', 'khimki', 'digital', 'chekhov', 'yakutsk', 'yaroslavl']


In [97]:
shoploc_traveling_idx = shop_loc_names_translated.index('traveling')  # convert to 'export'
shoploc_onlinestore_idx = shop_loc_names_translated.index('online store')  # convert to 'online'
shoploc_digital_idx = shop_loc_names_translated.index('digital')  # convert to 'online'

shop_loc_names_translated[shoploc_traveling_idx] = 'export'
shop_loc_names_translated[shoploc_onlinestore_idx] = 'online'
shop_loc_names_translated[shoploc_digital_idx] = 'online'

In [100]:
shop_name_loc_mapped = dict(zip(shop_loc_names_uniques,
                               shop_loc_names_translated))
train['shop_loc_name_eng'] = train['shop_loc_name'].map(shop_name_loc_mapped)
train.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price_month,date_month,date_year,item_category_id,item_category_name_eng,shop_name,shop_loc_name,shop_loc_name_eng
0,0,30,1,31.0,265.0,02,2013,40,cinema - dvd,"!Якутск Орджоникидзе, 56 фран",!Якутск,yakutsk
1,0,31,1,11.0,434.0,02,2013,37,cinema - blu-ray,"!Якутск Орджоникидзе, 56 фран",!Якутск,yakutsk
2,0,32,0,6.0,221.0,01,2013,40,cinema - dvd,"!Якутск Орджоникидзе, 56 фран",!Якутск,yakutsk
3,0,32,1,10.0,221.0,02,2013,40,cinema - dvd,"!Якутск Орджоникидзе, 56 фран",!Якутск,yakutsk
4,0,33,0,3.0,347.0,01,2013,37,cinema - blu-ray,"!Якутск Орджоникидзе, 56 фран",!Якутск,yakutsk


### Get additional information from the name of the city (location) using `web scraping`
SOURCE TO CRAWL : https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia_by_population

* HTML PATTERN
```html
<tr>
<th align="center">1</th>
<td><i><b><a href="/wiki/Moscow" title="Moscow">Moscow</a></b></i></td>
<td><span lang="ru" xml:lang="ru">Москва</span></td>
<td><span class="flagicon"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Flag_of_Moscow.svg/23px-Flag_of_Moscow.svg.png" width="23" height="15" class="thumbborder" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Flag_of_Moscow.svg/35px-Flag_of_Moscow.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Flag_of_Moscow.svg/45px-Flag_of_Moscow.svg.png 2x" data-file-width="1200" data-file-height="800" />&#160;</span><a href="/wiki/Moscow" title="Moscow">Moscow (federal city)</a><sup id="cite_ref-3" class="reference"><a href="#cite_note-3">[3]</a></sup></td>
<td><a href="/wiki/Central_Federal_District" title="Central Federal District">Central</a></td>
<td align="right" rowspan="1" style="background-color:#F9F9F9;">12,228,685</td>
<td align="right" rowspan="1" style="background-color:#F9F9F9;">11,503,501</td>
<td align="right" rowspan="1" style="background-color:#F9F9F9;"><span style="display:none" class="sortkey">7000630402866049210♠</span><span style="color:green">+6.30%</span></td>
</tr>
<tr>
<th align="center">2</th>
<td><i><b><a href="/wiki/Saint_Petersburg" title="Saint Petersburg">Saint Petersburg</a></b></i></td>
<td><span lang="ru" xml:lang="ru">Санкт-Петербург</span></td>
<td><span class="flagicon"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Flag_of_Saint_Petersburg_Russia.svg/23px-Flag_of_Saint_Petersburg_Russia.svg.png" width="23" height="15" class="thumbborder" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Flag_of_Saint_Petersburg_Russia.svg/35px-Flag_of_Saint_Petersburg_Russia.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Flag_of_Saint_Petersburg_Russia.svg/45px-Flag_of_Saint_Petersburg_Russia.svg.png 2x" data-file-width="1200" data-file-height="800" />&#160;</span><a href="/wiki/Saint_Petersburg" title="Saint Petersburg">Saint Petersburg (federal city)</a><sup id="cite_ref-4" class="reference"><a href="#cite_note-4">[4]</a></sup></td>
<td><a href="/wiki/Northwestern_Federal_District" title="Northwestern Federal District">Northwest</a></td>
<td align="right" rowspan="1" style="background-color:#F9F9F9;">5,281,579</td>
<td align="right" rowspan="1" style="background-color:#F9F9F9;">4,879,566</td>
<td align="right" rowspan="1" style="background-color:#F9F9F9;"><span style="display:none" class="sortkey">7000823870401588990♠</span><span style="color:green">+8.24%</span></td>
</tr>
```

### What to crawl for each city

* `shop_loc_population`
* `shop_loc_latitude`
* `shop_loc_longitude`

In [183]:
# get the information above for each city except for 'online' and 'export' values.

In [101]:
import requests
from bs4 import BeautifulSoup

In [102]:
print(shop_loc_names_uniques)

['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж', 'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга', 'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород', 'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев', 'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов', 'Якутск', 'Ярославль']


In [103]:
print(shop_name_loc_mapped)

{'!Якутск': 'yakutsk', 'Адыгея': 'adygea', 'Балашиха': 'balashiha', 'Волжский': 'volzhsky', 'Вологда': 'vologda', 'Воронеж': 'voronezh', 'Выездная': 'export', 'Жуковский': 'zhukovsky', 'Интернет-магазин': 'online', 'Казань': 'kazan', 'Калуга': 'kaluga', 'Коломна': 'kolomna', 'Красноярск': 'krasnoyarsk', 'Курск': 'kursk', 'Москва': 'moscow', 'Мытищи': 'mytischi', 'Н.Новгород': 'n.novgorod', 'Новосибирск': 'novosibirsk', 'Омск': 'omsk', 'РостовНаДону': 'rostovnadonu', 'СПб': 'st. petersburg', 'Самара': 'samara', 'Сергиев': 'sergiev', 'Сургут': 'surgut', 'Томск': 'tomsk', 'Тюмень': 'tyumen', 'Уфа': 'ufa', 'Химки': 'khimki', 'Цифровой': 'online', 'Чехов': 'chekhov', 'Якутск': 'yakutsk', 'Ярославль': 'yaroslavl'}


First five rows of wikipedia table we're scraping
![wiki-table](data/wiki-table.png)

In [104]:
url = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia_by_population'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
table_classes ={"class": ["sortable", "plainrowheaders"]}
wikitable = soup.findAll("table", table_classes)[0]

#print(wikitables)

In [105]:
# column names of wiki table

wikitable.findAll("tr")[0].text.split('\n')[1:-1]

['Rank (2017)',
 'City/town',
 'Russian',
 'Federal Subject',
 'Federal District',
 'Population',
 '(2017 estimate)[1]',
 'Population',
 '(2010 Census)[2]',
 'Change']

In [106]:
# values in the first row of wiki table
wikitable.findAll("tr")[1].text.split('\n')[1:-1]

['1',
 'Moscow',
 'Москва',
 '\xa0Moscow (federal city)[3]',
 'Central',
 '12,228,685',
 '11,503,501',
 '7000630402866049210♠+6.30%']

In [107]:
wikitable_cols = [
    [],[],[],[],[],[],[],[]
]

for i, row in enumerate(wikitable.findAll("tr")):
    if i != 0:
        row_values = row.text.split('\n')[1:-1]
        #print(row_values)

        for j, val in enumerate(row_values):
            wikitable_cols[j].append(str(val))

In [108]:
wiki_ru_table = pd.DataFrame({'rank_2017' : wikitable_cols[0],
                             'name_en' : wikitable_cols[1],
                             'name_ru' : wikitable_cols[2],
                             'fed_sub' : wikitable_cols[3],
                             'fed_dist' : wikitable_cols[4],
                             'pop_2017_est' : wikitable_cols[5],
                             'pop_2010_cen' : wikitable_cols[6],
                             'change_perc' : wikitable_cols[7]})

wiki_ru_table.head()

,change_perc,fed_dist,fed_sub,name_en,name_ru,pop_2010_cen,pop_2017_est,rank_2017
0,7000630402866049210♠+6.30%,Central,Moscow (federal city)[3],Moscow,Москва,"11,503,501","12,228,685",1
1,7000823870401588990♠+8.24%,Northwest,Saint Petersburg (federal city)[4],Saint Petersburg,Санкт-Петербург,"4,879,566","5,281,579",2
2,7000876408138671720♠+8.76%,Siberia,Novosibirsk Oblast,Novosibirsk,Новосибирск,"1,473,754","1,602,915",3
3,7000783406382707600♠+7.83%,Ural,Sverdlovsk Oblast,Yekaterinburg,Екатеринбург,"1,349,772","1,455,514",4
4,6999883322578659040♠+0.88%,Volga,Nizhny Novgorod Oblast,Nizhny Novgorod,Нижний Новгород,"1,250,619","1,261,666",5


In [109]:
train.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price_month,date_month,date_year,item_category_id,item_category_name_eng,shop_name,shop_loc_name,shop_loc_name_eng
0,0,30,1,31.0,265.0,02,2013,40,cinema - dvd,"!Якутск Орджоникидзе, 56 фран",!Якутск,yakutsk
1,0,31,1,11.0,434.0,02,2013,37,cinema - blu-ray,"!Якутск Орджоникидзе, 56 фран",!Якутск,yakutsk
2,0,32,0,6.0,221.0,01,2013,40,cinema - dvd,"!Якутск Орджоникидзе, 56 фран",!Якутск,yakutsk
3,0,32,1,10.0,221.0,02,2013,40,cinema - dvd,"!Якутск Орджоникидзе, 56 фран",!Якутск,yakutsk
4,0,33,0,3.0,347.0,01,2013,37,cinema - blu-ray,"!Якутск Орджоникидзе, 56 фран",!Якутск,yakutsk


In [110]:
wiki_ru_table['name_en_lowered'] = wiki_ru_table['name_en'].apply(lambda x: x.lower())

In [112]:
joined_wiki_cityname_uniques = ' '.join(wiki_ru_table.name_en_lowered.unique())
train_wiki_cityname_mapped = {}

for train_cityname in train.shop_loc_name_eng.unique():
    
    find_all_in_wiki = re.findall(train_cityname, joined_wiki_cityname_uniques)
    
    if (train_cityname in ['online', 'export']) or find_all_in_wiki:
        train_wiki_cityname_mapped[train_cityname] = train_cityname
        
    else:
        train_wiki_cityname_mapped[train_cityname] = 'NOT FOUND'
        

In [113]:
train_wiki_cityname_mapped

{'adygea': 'NOT FOUND',
 'balashiha': 'NOT FOUND',
 'chekhov': 'chekhov',
 'export': 'export',
 'kaluga': 'kaluga',
 'kazan': 'kazan',
 'khimki': 'khimki',
 'kolomna': 'kolomna',
 'krasnoyarsk': 'krasnoyarsk',
 'kursk': 'kursk',
 'moscow': 'moscow',
 'mytischi': 'NOT FOUND',
 'n.novgorod': 'NOT FOUND',
 'novosibirsk': 'novosibirsk',
 'omsk': 'omsk',
 'online': 'online',
 'rostovnadonu': 'NOT FOUND',
 'samara': 'samara',
 'sergiev': 'NOT FOUND',
 'st. petersburg': 'NOT FOUND',
 'surgut': 'surgut',
 'tomsk': 'tomsk',
 'tyumen': 'tyumen',
 'ufa': 'ufa',
 'vologda': 'vologda',
 'volzhsky': 'volzhsky',
 'voronezh': 'voronezh',
 'yakutsk': 'yakutsk',
 'yaroslavl': 'yaroslavl',
 'zhukovsky': 'zhukovsky'}

In [114]:
# manually find the city names not matched except for 'adgyea' - just a subtle difference in text

# 'adygea' is the name of federal subject in Russia - not a name of city
# Full name in Russian for 'adygea' translated loc name = 'Адыгея ТЦ "Мега"'
print([shopname for shopname in train.shop_name.unique() if shopname.startswith('Адыгея')][0])

# google the full name directly and get the city name - 'urgenevskoye Shosse, 27, Novaya Adygeya, Krasnodar Krai'
# >>> 'krasnodar' found in wikitable.

# finish the dictionary for mapping.
train_wiki_cityname_mapped = {'adygea': 'krasnodar',
                             'balashiha': 'balashikha',
                             'chekhov': 'chekhov',
                             'export': 'export',
                             'kaluga': 'kaluga',
                             'kazan': 'kazan',
                             'khimki': 'khimki',
                             'kolomna': 'kolomna',
                             'krasnoyarsk': 'krasnoyarsk',
                             'kursk': 'kursk',
                             'moscow': 'moscow',
                             'mytischi': 'mytishchi',
                             'n.novgorod': 'nizhny novgorod',
                             'novosibirsk': 'novosibirsk',
                             'omsk': 'omsk',
                             'online': 'online',
                             'rostovnadonu': 'rostov-on-don',
                             'samara': 'samara',
                             'sergiev': 'sergiyev posad',
                             'st. petersburg': 'saint petersburg',
                             'surgut': 'surgut',
                             'tomsk': 'tomsk',
                             'tyumen': 'tyumen',
                             'ufa': 'ufa',
                             'vologda': 'vologda',
                             'volzhsky': 'volzhsky',
                             'voronezh': 'voronezh',
                             'yakutsk': 'yakutsk',
                             'yaroslavl': 'yaroslavl',
                             'zhukovsky': 'zhukovsky'
}

Адыгея ТЦ "Мега"


In [115]:
# Using the dictionary above, we created a modified version of city(location) name column.
train['shop_loc_name_eng_corrected'] = train['shop_loc_name_eng'].map(train_wiki_cityname_mapped)

In [116]:
# write a function to additional info from wiki table by city

def grab_city_info(cityname):
    
    if cityname in ['online', 'export']:
        return None, None, None, None, None
    
    city_data = wiki_ru_table[wiki_ru_table['name_en_lowered'] == cityname]
    
    city_fed_dist = city_data['fed_dist'].values[0]
    city_pop_2010 = city_data['pop_2010_cen'].values[0]
    city_pop_2017_est = city_data['pop_2017_est'].values[0]
    city_pop_rank_2017 = city_data['rank_2017'].values[0]
    city_pop_change_2010_2017 = city_data['change_perc'].values[0]
    
    return city_fed_dist, city_pop_2010, city_pop_2017_est, city_pop_rank_2017, city_pop_change_2010_2017

In [117]:
# clean data before executing 'grab_city_info()' function
wiki_ru_table.head()

,change_perc,fed_dist,fed_sub,name_en,name_ru,pop_2010_cen,pop_2017_est,rank_2017,name_en_lowered
0,7000630402866049210♠+6.30%,Central,Moscow (federal city)[3],Moscow,Москва,"11,503,501","12,228,685",1,moscow
1,7000823870401588990♠+8.24%,Northwest,Saint Petersburg (federal city)[4],Saint Petersburg,Санкт-Петербург,"4,879,566","5,281,579",2,saint petersburg
2,7000876408138671720♠+8.76%,Siberia,Novosibirsk Oblast,Novosibirsk,Новосибирск,"1,473,754","1,602,915",3,novosibirsk
3,7000783406382707600♠+7.83%,Ural,Sverdlovsk Oblast,Yekaterinburg,Екатеринбург,"1,349,772","1,455,514",4,yekaterinburg
4,6999883322578659040♠+0.88%,Volga,Nizhny Novgorod Oblast,Nizhny Novgorod,Нижний Новгород,"1,250,619","1,261,666",5,nizhny novgorod


In [118]:
wiki_ru_table['change_perc'] = wiki_ru_table['change_perc'].apply(lambda x: x.split('♠')[1].split('%')[0])

In [119]:
# function test
print(grab_city_info('moscow'))
print(grab_city_info('kursk'))

('Central', '11,503,501', '12,228,685', '1', '+6.30')
('Central', '415,159', '449,063', '42', '+8.17')


In [120]:
# create a dictionary to extract the city info

train_city_name_unique = train['shop_loc_name_eng_corrected'].unique().tolist()
grabbed_city_info = [grab_city_info(city) for city in train_city_name_unique]

grabbed_city_info_dict = {k:v for (k,v) in zip(train_city_name_unique,
                                              grabbed_city_info)}

grabbed_city_info_dict

{'balashikha': ('Central', '215,494', '450,771', '41', '+109.18'),
 'chekhov': ('Central', '60,720', '70,548', '226', '+16.19'),
 'export': (None, None, None, None, None),
 'kaluga': ('Central', '324,698', '341,892', '56', '+5.30'),
 'kazan': ('Volga', '1,143,535', '1,231,878', '6', '+7.73'),
 'khimki': ('Central', '207,425', '244,668', '78', '+17.95'),
 'kolomna': ('Central', '144,589', '144,125', '121', '−0.32'),
 'krasnodar': ('South', '744,995', '881,476', '16', '+18.32'),
 'krasnoyarsk': ('Siberia', '973,826', '1,082,933', '12', '+11.20'),
 'kursk': ('Central', '415,159', '449,063', '42', '+8.17'),
 'moscow': ('Central', '11,503,501', '12,228,685', '1', '+6.30'),
 'mytishchi': ('Central', '173,160', '205,397', '93', '+18.62'),
 'nizhny novgorod': ('Volga', '1,250,619', '1,261,666', '5', '+0.88'),
 'novosibirsk': ('Siberia', '1,473,754', '1,602,915', '3', '+8.76'),
 'omsk': ('Siberia', '1,154,116', '1,178,391', '8', '+2.10'),
 'online': (None, None, None, None, None),
 'rostov-on-d

In [121]:
train['shop_loc_fed'] = train['shop_loc_name_eng_corrected'].apply(lambda x: grabbed_city_info_dict[x][0])
train['shop_loc_pop_2010'] = train['shop_loc_name_eng_corrected'].apply(lambda x: grabbed_city_info_dict[x][1])
train['shop_loc_pop_2017'] = train['shop_loc_name_eng_corrected'].apply(lambda x: grabbed_city_info_dict[x][2])
train['shop_loc_pop_rank_2017'] = train['shop_loc_name_eng_corrected'].apply(lambda x: grabbed_city_info_dict[x][3])
train['shop_loc_pop_change_2010_2017'] = train['shop_loc_name_eng_corrected'].apply(lambda x: grabbed_city_info_dict[x][4])

### Back to `category_name` - 
### Split the category name into two levels

In [122]:
item_category_name_eng_uniques = pd.Series(train['item_category_name_eng'].unique().tolist())
item_category_split_names = [tup for tup in item_category_name_eng_uniques.apply(lambda x: tuple(x.split(' - ')))]

In [123]:
item_category_split_mapped = {k:v for (k,v) in zip(item_category_name_eng_uniques,
                                                  item_category_split_names)}

In [124]:
train['item_category_name_eng_1'] = train['item_category_name_eng'].apply(lambda x: item_category_split_mapped[x][0])
train['item_category_name_eng_2'] = train['item_category_name_eng'].apply(lambda x: item_category_split_mapped[x][1]\
                                                                         if len(item_category_split_mapped[x]) == 2\
                                                                          else None)

### Drop unnecessary columns
* `item_category_name_eng`
* `shop_name`
* `shop_loc_name`
* `shop_loc_name_eng`

In [125]:
train = train.drop(['item_category_name_eng',
                   'shop_name',
                   'shop_loc_name',
                   'shop_loc_name_eng'], axis = 1)

train.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price_month,date_month,date_year,item_category_id,shop_loc_name_eng_corrected,shop_loc_fed,shop_loc_pop_2010,shop_loc_pop_2017,shop_loc_pop_rank_2017,shop_loc_pop_change_2010_2017,item_category_name_eng_1,item_category_name_eng_2
0,0,30,1,31.0,265.0,02,2013,40,yakutsk,Far East,"269,601","307,911",64,+14.21,cinema,dvd
1,0,31,1,11.0,434.0,02,2013,37,yakutsk,Far East,"269,601","307,911",64,+14.21,cinema,blu-ray
2,0,32,0,6.0,221.0,01,2013,40,yakutsk,Far East,"269,601","307,911",64,+14.21,cinema,dvd
3,0,32,1,10.0,221.0,02,2013,40,yakutsk,Far East,"269,601","307,911",64,+14.21,cinema,dvd
4,0,33,0,3.0,347.0,01,2013,37,yakutsk,Far East,"269,601","307,911",64,+14.21,cinema,blu-ray


In [126]:
# rename olumns
to_clean_columns = {
    'date_month':'date_month',
    'date_year':'date_year',
    'date_block_num':'date_block_num',
    'item_id':'item_id',
    'shop_id':'shop_id',
    'item_category_id':'item_cateogory_id',
    'item_price_month':'item_price_month',
    'item_cnt_month':'item_cnt_month',
    'item_category_name_eng_1':'item_first_category',
    'item_category_name_eng_2':'item_second_category',
    'shop_loc_name_eng_corrected':'shop_loc',
    'shop_loc_fed':'shop_loc_dist',
    'shop_loc_pop_2010':'shop_loc_pop_2010',
    'shop_loc_pop_2017':'shop_loc_pop_2017',
    'shop_loc_pop_rank_2017':'shop_loc_pop_rank_2017',
    'shop_loc_pop_change_2010_2017':'shop_loc_pop_change_2010_2017'
}


train = train.rename(columns=to_clean_columns)

## Export preprocessed data to `csv`
### In the following notebook, we will optimize the data before EDA & feature-engineering.

In [130]:
train['shop_loc_pop_2010'] = train['shop_loc_pop_2010'].apply(lambda x: re.sub(r',', '', x) if x is not None else None)
train['shop_loc_pop_2017'] = train['shop_loc_pop_2017'].apply(lambda x: re.sub(r',', '', x) if x is not None else None)

In [131]:
train['shop_loc_pop_change_2010_2017'] =\
            100 * (train['shop_loc_pop_2017'].astype('float') /\
                                                train['shop_loc_pop_2010'].astype('float') - 1).apply(lambda x:\
                                                                                                     round(x, 3))

In [132]:
train.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price_month,date_month,date_year,item_cateogory_id,shop_loc,shop_loc_dist,shop_loc_pop_2010,shop_loc_pop_2017,shop_loc_pop_rank_2017,shop_loc_pop_change_2010_2017,item_first_category,item_second_category
0,0,30,1,31.0,265.0,02,2013,40,yakutsk,Far East,269601,307911,64,14.2,cinema,dvd
1,0,31,1,11.0,434.0,02,2013,37,yakutsk,Far East,269601,307911,64,14.2,cinema,blu-ray
2,0,32,0,6.0,221.0,01,2013,40,yakutsk,Far East,269601,307911,64,14.2,cinema,dvd
3,0,32,1,10.0,221.0,02,2013,40,yakutsk,Far East,269601,307911,64,14.2,cinema,dvd
4,0,33,0,3.0,347.0,01,2013,37,yakutsk,Far East,269601,307911,64,14.2,cinema,blu-ray


In [133]:
# export in the local data folder
train.to_csv('data/sales_train_opt_ver1.csv', index=False)